In [9]:
import tensorflow as tf
import numpy as np
import psycopg2
import pandas as pd

In [37]:
### load dataset
connection = psycopg2.connect(user = "postgres",
                                      password = "root",
                                      host = "localhost",
                                      port = "5432",
                                      database='postgres')

# usage: specify file location, sample size and seed(used by random) 
filepath = '../Data_git_ignore/clean_csv/article_clean.csv' # 250 rows of FakeNewsCorpus
f = 1.0                                            # desired frac
seed = 1                                           # seed used by Pseudorandom number generator

df = pd.read_sql_query("SELECT content, type_id FROM fakenews.article LIMIT 15000", connection).sample(frac=f, random_state=seed)
#df["content"] = df["content"].astype(str)
# c*reate type_id
#df['type_id'] = df.groupby(['type']).ngroup()

df

,content,type_id
7576,from reason magazine which really ought to loo...,2
10509,christians are being annihilated — wiped from ...,3
4253,i wrote about the nfl ratings about a month ag...,1
5150,the pm and the opposition leader have both exp...,1
506,headline bitcoin blockchain searches exceed tr...,0
...,...,...
905,weekly crude oil gold silver coffee and sugar ...,0
5192,NUM k shares share the physician white coat c...,1
12172,natural news brevard county sheriff wayne ivey...,4
235,headline bitcoin blockchain searches exceed tr...,0


In [82]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

# #df['content'] = df['content'].apply(' '.join)

#df_td_idf = pd.DataFrame (v.fit_transform(df["content"]).toarray())

In [39]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x, df["type_id"], test_size=0.2, random_state=0)

In [35]:
x = x.reindex(np.random.permutation(x.index))

#x = x.reindex(np.random.permutation(X.index))

#np.random.shuffle(x)

#test_proportion of 3 means 1/3 so 33% test and 67% train
def shuffle(matrix, target, test_proportion):
    ratio = int(matrix.shape[0]/test_proportion) #should be int
    X_train = matrix[ratio:,:]
    X_test =  matrix[:ratio,:]
    Y_train = target[ratio:,:]
    Y_test =  target[:ratio,:]
    return X_train, X_test, Y_train, Y_test

X_train, X_test, Y_train, Y_test = shuffle(x, df["type_id"].values, 3)

AttributeError: reindex not found

In [44]:
print(x.shape[0])

print(len(df["type_id"].values))

100
10000


In [59]:
### SELECT TRAIN, TEST, VALIDATE ###

df = df[['type_id', 'content']]

# This can generate a dataset with random purmutation and a max size for each type(can be smaller if desired max is not possible)

# max size for type
max_size = 1000
# traning_set ratio - splits data into traning=ratio,  test and validate=(1-ratio)/2 ex. train=80%, test=10%, validate=10%
ratio=0.7

# Labels to include - ['fake', 'satire', 'bias', 'conspiracy', 'state', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable'] - all labels
# use_types = ['fake', 'satire', 'bias', 'conspiracy', 'junksci', 'hate', 'clickbait', 'unreliable', 'political', 'reliable'] # <- string labes
use_types = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9] # <- int labels

# Random seed
rnd = 1

# initialize dataframes
train    = pd.DataFrame(columns = df.columns)
test     = pd.DataFrame(columns = df.columns)
validate = pd.DataFrame(columns = df.columns)

# add type to test splits
for t in use_types:

    # type size
    type_size = df['type_id'].loc[df['type_id'] == t].value_counts().min()

    # set size of type slice
    if type_size < max_size:
        tmp = df.loc[df['type_id'] == t].sample(n = type_size, random_state=rnd)
    else:
        tmp = df.loc[df['type_id'] == t].sample(n = max_size, random_state=rnd)

    # split current type
    train_tmp, test_tmp, validate_tmp = np.split(tmp, [int(ratio * len(tmp)), int(((1-ratio)/2 + ratio) * len(tmp))])

    # add tmp to dataframes
    train    = pd.concat([train, train_tmp])
    test     = pd.concat([test, test_tmp])
    validate = pd.concat([validate, validate_tmp])
    
    # print split shape
    print("=>", t, tmp.shape, train_tmp.shape, validate_tmp.shape, validate_tmp.shape)

print("\n[Final split]\ntrain, test, validate ==>", train.shape, test.shape, validate.shape)

# Create X, y split for all sets
X_train    = train['content']
y_train    = train['type_id']
X_test     = test['content']
y_test     = test['type_id']
X_validate = validate['content']
y_validate = validate['type_id']

# Free memory
del train
del test
del validate

=> 0 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 1 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 2 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 3 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 4 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 5 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 6 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 7 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 8 (1000, 2) (700, 2) (150, 2) (150, 2)
=> 9 (1000, 2) (700, 2) (150, 2) (150, 2)

[Final split]
train, test, validate ==> (7000, 2) (1500, 2) (1500, 2)


In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
v = TfidfVectorizer()

x = v.fit_transform(df['content'])

In [48]:
# laver modellen
model = tf.keras.models.Sequential()
# tilføjer et input på modellen
model.add(tf.keras.layers.Flatten())

# relu er default aktiverings funktion. Lav den om hvis resultatet ikke er godt nok
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))
# jeg tilføjer 2 lag til netwærket. Dette er fordi det er en simpel opgave
model.add(tf.keras.layers.Dense(20, activation=tf.nn.relu))

#antallet (10) er antal output. Det er 10 tal i datasettet derfor skal der være et 10 tal
model.add(tf.keras.layers.Dense(11, activation=tf.nn.softmax))

# Dette er den mest komplexe del. adam er goto. Hvis der kun er 2 løsninger så brug binary_categorical_crossentropy eller binary_crossentropy i stedet for sparse_categorical_crossentropy
model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

In [41]:
x.toarray()

MemoryError: 

In [60]:
y_train.values[:7001]

array([4, 2, 2, ..., 1, 7, 1], dtype=int64)

In [49]:
model.fit(X_train.toarray(), y_train.values, epochs=3)

MemoryError: 

In [80]:
# dette for for at trække om modellen er overfittet. val_loss og val_acc skal minde meget om det man får i fittet. 
#Det er ok at loss og acc er lidt højre på testen. De må ikke være fortæt eller for længt væk fra hinaden
val_loss, val_acc = model.evaluate(X_test.toarray(),y_test.values)
print(val_loss, val_acc)

20/20 [==============================] - 0s 9ms/sample - loss: 0.6735 - accuracy: 1.0000
0.6735154390335083 1.0
